In [31]:
import numpy as np
import rasterio
import pandas as pd
import os

# 读取城市列表
cities_df = pd.read_csv('CitiesDB_new.csv')
print(f"共有 {len(cities_df)} 个城市需要处理")

# 设置阈值
threshold = 0.34

# 存储结果的列表
results = []

# 遍历所有城市
for index, row in cities_df.iterrows():
    city_name = row.iloc[0]  # 假设第一列是城市名称，根据实际情况调整
    
    # print(f"正在处理城市 {index+1}/{len(cities_df)}: {city_name}")
    
    try:
        # 构建文件路径
        image_2016_path = f"images/{city_name}_2016.tif"
        image_2023_path = f"images/{city_name}_2023.tif"
        
        # 检查文件是否存在
        if not os.path.exists(image_2016_path):
            print(f"  警告: {image_2016_path} 不存在")
            continue
        if not os.path.exists(image_2023_path):
            print(f"  警告: {image_2023_path} 不存在")
            continue
            
        # 读取2016年图像
        with rasterio.open(image_2016_path) as src:
            image_2016 = src.read(2)  # 第二个band
            
        # 读取2023年图像
        with rasterio.open(image_2023_path) as src:
            image_2023 = src.read(2)  # 第二个band
            
        # 阈值处理
        binary_2016 = (image_2016 > threshold).astype(int)
        binary_2023 = (image_2023 > threshold).astype(int)
        
        # 统计2016年
        count_0_2016 = np.sum(binary_2016 == 0)
        count_1_2016 = np.sum(binary_2016 == 1)
        total_2016 = binary_2016.size
        
        # 统计2023年
        count_0_2023 = np.sum(binary_2023 == 0)
        count_1_2023 = np.sum(binary_2023 == 1)
        total_2023 = binary_2023.size
        
        # 计算百分比
        pct_0_2016 = (count_0_2016 / total_2016) * 100
        pct_1_2016 = (count_1_2016 / total_2016) * 100
        pct_0_2023 = (count_0_2023 / total_2023) * 100
        pct_1_2023 = (count_1_2023 / total_2023) * 100
        
        # 计算变化
        change_1_count = count_1_2023 - count_1_2016
        change_1_pct = (change_1_count / count_1_2016) * 100 if count_1_2016 > 0 else 0
        
        # 存储结果
        result = {
            'City': city_name,
            'Total_Pixels_2016': total_2016,
            'Count_0_2016': count_0_2016,
            'Count_1_2016': count_1_2016,
            'Pct_0_2016': pct_0_2016,
            'Pct_1_2016': pct_1_2016,
            'Total_Pixels_2023': total_2023,
            'Count_0_2023': count_0_2023,
            'Count_1_2023': count_1_2023,
            'Pct_0_2023': pct_0_2023,
            'Pct_1_2023': pct_1_2023,
            'Change_1_Count': change_1_count,
            'Change_1_Percent': change_1_pct
        }
        
        results.append(result)
        
        # 打印当前城市的结果
        # print(f"  2016: 0={count_0_2016:,}({pct_0_2016:.1f}%), 1={count_1_2016:,}({pct_1_2016:.1f}%)")
        # print(f"  2023: 0={count_0_2023:,}({pct_0_2023:.1f}%), 1={count_1_2023:,}({pct_1_2023:.1f}%)")
        # print(f"  变化: 1的像素 {change_1_count:+,} ({change_1_pct:+.2f}%)")
        
    except Exception as e:
        print(f"  错误处理城市 {city_name}: {str(e)}")
        continue

# 转换为DataFrame并保存结果
results_df = pd.DataFrame(results)
output_file = 'threshold_analysis_results.csv'
results_df.to_csv(output_file, index=False)

print(f"\n=== 处理完成 ===")
print(f"成功处理了 {len(results_df)} 个城市")
print(f"结果已保存到: {output_file}")

# 显示汇总统计
if len(results_df) > 0:
    print(f"\n=== 汇总统计 ===")
    print(f"2016年平均1的占比: {results_df['Pct_1_2016'].mean():.2f}%")
    print(f"2023年平均1的占比: {results_df['Pct_1_2023'].mean():.2f}%")
    print(f"平均变化: {results_df['Change_1_Percent'].mean():+.2f}%")
    print(f"1的占比增长最大的城市: {results_df.loc[results_df['Change_1_Percent'].idxmax(), 'City']} ({results_df['Change_1_Percent'].max():+.2f}%)")
    print(f"1的占比减少最大的城市: {results_df.loc[results_df['Change_1_Percent'].idxmin(), 'City']} ({results_df['Change_1_Percent'].min():+.2f}%)")
    
    # 显示前5个结果作为预览
    print(f"\n=== 前5个城市结果预览 ===")
    print(results_df.head().to_string(index=False))

共有 104 个城市需要处理

=== 处理完成 ===
成功处理了 104 个城市
结果已保存到: threshold_analysis_results.csv

=== 汇总统计 ===
2016年平均1的占比: 14.97%
2023年平均1的占比: 17.05%
平均变化: +17.37%
1的占比增长最大的城市: Monrovia (+54.55%)
1的占比减少最大的城市: Mumbai (+1.14%)

=== 前5个城市结果预览 ===
        City  Total_Pixels_2016  Count_0_2016  Count_1_2016  Pct_0_2016  Pct_1_2016  Total_Pixels_2023  Count_0_2023  Count_1_2023  Pct_0_2023  Pct_1_2023  Change_1_Count  Change_1_Percent
Antananarivo           16008001      14878123       1129878   92.941792    7.058208           16008001      14600758       1407243   91.209127    8.790873          277365         24.548226
     Abidjan           16008001      13966725       2041276   87.248402   12.751598           16008001      13327602       2680399   83.255879   16.744121          639123         31.309975
     Yaounde           16008001      14640706       1367295   91.458677    8.541323           16008001      14037616       1970385   87.691249   12.308751          603090         44.108258
      Lusaka  

In [1]:
import os
import pickle
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import cupy as cp
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import gaussian_kde
from scipy.ndimage import uniform_filter, convolve
from shapely.geometry import Point, Polygon
import alphashape
from scipy.spatial.distance import cdist
import pyproj
from shapely.ops import transform
from functools import partial
import math
from pathlib import Path
from rasterio.transform import xy  # Import the xy function

def read_tiff(file_path):
    """Read TIFF file with enhanced debugging"""
    print(f"[DEBUG] Reading TIFF file: {file_path}")
    try:
        with rasterio.open(file_path) as src:
            data = src.read()
            meta = src.meta.copy()
            transform = src.transform
            crs = src.crs
            bounds = src.bounds  # Additional metadata

            print(f"[DEBUG] Bands: {src.count}")
            print(f"[DEBUG] Shape: {data.shape}")
            print(f"[DEBUG] CRS: {crs}")
            print(f"[DEBUG] Bounds: {bounds}")

            return data, meta, transform, crs, bounds
    except RasterioIOError as e:
        print(f"[ERROR] TIFF reading failed: {e}")
        return None

def convert_to_projected_coords(points, source_crs):
    """
    Convert points from source CRS to a suitable UTM projection
    
    Parameters:
    -----------
    points : numpy.ndarray
        Array of shape (n, 2) containing x,y coordinates
    source_crs : str or CRS
        The source coordinate reference system
    
    Returns:
    --------
    projected_points : list
        List of projected coordinates as tuples
    utm_crs : CRS
        The target UTM CRS
    """
    # Check if points array is empty using NumPy's size attribute
    if points.size == 0:
        return [], None
        
    # Convert source_crs to pyproj CRS if it's a string
    if isinstance(source_crs, str):
        source_crs = pyproj.CRS.from_string(source_crs)
    
    # Calculate the centroid of points to determine the appropriate UTM zone
    mean_lon = np.mean(points[:, 0])
    mean_lat = np.mean(points[:, 1])
    
    # Determine UTM zone based on the centroid
    utm_crs = pyproj.CRS.from_dict({
        'proj': 'utm',
        'zone': int((mean_lon + 180) / 6) + 1,
        'south': mean_lat < 0
    })
    
    # Create the transformer
    transformer = pyproj.Transformer.from_crs(source_crs, utm_crs, always_xy=True)
    
    # Transform all points
    x, y = transformer.transform(points[:, 0], points[:, 1])
    
    # Convert to a list of tuples for alphashape
    projected_points = list(zip(x, y))
    
    return projected_points, utm_crs

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the Haversine distance between two points on Earth"""
    # Convert coordinates to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of Earth in kilometers
    r = 6371
    
    return c * r

def create_sliding_window_dataframe(data, transform, city_center, crs, output_dir="pixel_data",threshold=0.5):
    """
    Create a DataFrame using 3x3 sliding window analysis with vectorized operations
    Starting from row 1, col 1 (0-indexed) to ensure full 3x3 neighborhood
    """
    print(f"[DEBUG] Creating sliding window DataFrame with 3x3 windows using vectorized operations")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Extract bands
    building_height = data[1]  # Height band (band 2, 0-indexed as 1)
    building_presence = data[2]  # Building presence band (band 3, 0-indexed as 2)

    print(f"[DEBUG] Image shape: {building_presence.shape}")
    print(f"[DEBUG] Building presence range: {building_presence.min()} to {building_presence.max()}")

    # Apply threshold to create building existence binary mask
    building_exist = (building_presence > threshold).astype(int)
    print(f"[DEBUG] Building existence pixels: {np.sum(building_exist)} out of {building_exist.size}")

    # Get image dimensions
    height, width = building_presence.shape

    print(f"[DEBUG] Starting vectorized 3x3 window processing...")
    
    # Create 3x3 kernel for convolution
    kernel_3x3 = np.ones((3, 3), dtype=np.float32)
    
    building_count_full = convolve(building_exist.astype(np.float32), kernel_3x3, mode='constant', cval=0.0)
    building_count_full = building_count_full.astype(int)
    
    # For average height calculation, we need to handle division by building count
    # Use convolution to compute sum in 3x3 windows efficiently
    height_sum_full = convolve((building_height * building_exist).astype(np.float32), kernel_3x3, mode='constant', cval=0.0)
    
    # Extract the valid region (excluding border pixels)
    building_counts = building_count_full[1:-1, 1:-1]
    height_sums = height_sum_full[1:-1, 1:-1]
    
    # Calculate average heights (avoid division by zero warnings)
    avg_heights = np.divide(height_sums, building_counts, 
                           out=np.zeros_like(height_sums, dtype=float), 
                           where=building_counts > 0)
        
    print(f"[DEBUG] Vectorized window processing completed")
    
    # Generate coordinate grids for center pixels
    rows_range = np.arange(1, height - 1)
    cols_range = np.arange(1, width - 1)
    row_grid, col_grid = np.meshgrid(rows_range, cols_range, indexing='ij')
    
    # Flatten grids for output
    center_rows = row_grid.flatten()
    center_cols = col_grid.flatten()
    building_counts_flat = building_counts.flatten()
    avg_heights_flat = avg_heights.flatten()
    
    print(f"[DEBUG] Converting pixel coordinates to geographic coordinates...")
    
    # Batch convert pixel coordinates to geographic coordinates
    center_xs, center_ys = xy(transform, center_rows, center_cols, offset='center')
    
    print(f"[DEBUG] Calculating distances to city center...")
    
    # Vectorized distance calculation using numpy broadcasting
    center_ys_array = np.array(center_ys)
    center_xs_array = np.array(center_xs)
    
    # Vectorized haversine distance calculation
    lat1, lon1 = np.radians(center_ys_array), np.radians(center_xs_array)
    lat2, lon2 = np.radians(city_center[0]), np.radians(city_center[1])
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distances = c * 6371  # Earth radius in km

    # Create DataFrame
    df_windows = pd.DataFrame({
        'center_row': center_rows,
        'center_col': center_cols,
        'center_longitude': center_xs_array,
        'center_latitude': center_ys_array,
        'building_count': building_counts_flat,  # Number of pixels with buildings (0-9)
        'avg_height': avg_heights_flat,  # Average height of building pixels
        'distance_to_center': distances  # Distance to city center in km
    })

    print(f"[DEBUG] Created DataFrame with {len(df_windows)} windows")
    
    # Print statistics
    print(f"[INFO] Window analysis statistics:")
    print(f"  - Building count range: {df_windows['building_count'].min()} to {df_windows['building_count'].max()}")
    print(f"  - Average height range: {df_windows['avg_height'].min():.2f} to {df_windows['avg_height'].max():.2f}")
    print(f"  - Distance range: {df_windows['distance_to_center'].min():.2f} to {df_windows['distance_to_center'].max():.2f} km")
    print(f"  - Windows with buildings: {len(df_windows[df_windows['building_count'] > 0])}")

    return df_windows

def process_city(tiff_file, city_name, year, city_center, t):
    """Process a single city's TIFF file with 3x3 sliding window analysis"""
    print(f"[INFO] Processing {city_name} {year} with 3x3 sliding window analysis")
    
    try:
        # Create output directories
        output_dirs = ["pixel_data", "visualizations"]
        for directory in output_dirs:
            os.makedirs(directory, exist_ok=True)
        
        # Read TIFF
        print("[DEBUG] Reading TIFF file...")
        result = read_tiff(tiff_file)
        if result is not None:
            data, meta, transform, crs, bounds = result
            # Proceed with processing the data
        else:
            # Handle the error case
            print("Failed to read the TIFF file.")
            return False
            # Create sliding window DataFrame
        df_windows = create_sliding_window_dataframe(data, transform, city_center, crs, threshold = t)
        
        # Save the windows dataframe
        windows_output_file = os.path.join("aggregated_data", f"{city_name}_{year}_3x3_windows.csv")
        df_windows.to_csv(windows_output_file, index=False)
        print(f"[DEBUG] Saved {len(df_windows)} windows to {windows_output_file}")
        
        print(f"[INFO] {city_name} {year} analysis completed successfully!")
        return True
        
    except Exception as e:
        print(f"[ERROR] An error occurred while processing {city_name} {year}: {e}")
        import traceback
        traceback.print_exc()
        return False

In [12]:
##生成cities列表
import pandas as pd

# 读取 CSV 文件
df = pd.read_csv("CitiesDB_new.csv")

# 获取第9到第18个城市（索引为8到17）
#subset = df.iloc[50:51] #Mumbai
#subset = df.iloc[0:1]
subset = df.iloc[50:104]

# 构建 cities 列表
cities = []

# 添加从 CSV 中提取的城市
for _, row in subset.iterrows():
    city = row["Name"]
    lon = row["Cx(lon)"]
    lat = row["Cy(lat)"]
    cities.append((city, [2016, 2023], (lat, lon)))

# 打印结果验证
for c in cities:
    print(c)

('Mumbai', [2016, 2023], (18.968949, 72.819119))
('Manila', [2016, 2023], (14.598171, 120.983366))
('Dhaka', [2016, 2023], (23.731382, 90.425371))
('Kolkata', [2016, 2023], (22.572363, 88.358792))
('Bangkok', [2016, 2023], (13.756876, 100.502114))
('Karachi', [2016, 2023], (24.849182, 67.002087))
('Bangalore', [2016, 2023], (12.972095, 77.594172))
('Ho Chi Minh City', [2016, 2023], (10.778004, 106.696322))
('Chennai', [2016, 2023], (13.082905, 80.274252))
('Hyderabad', [2016, 2023], (17.380835, 78.484755))
('Kuala Lumpur', [2016, 2023], (3.145446, 101.69502))
('Bandung', [2016, 2023], (-6.914929, 107.602485))
('Pune', [2016, 2023], (18.525565, 73.867503))
('Ahmedabad', [2016, 2023], (23.025975, 72.598625))
('Surat', [2016, 2023], (21.205076, 72.840369))
('Surabaya', [2016, 2023], (-7.243775, 112.738052))
('Jaipur', [2016, 2023], (26.912351, 75.819625))
('Phnom Penh', [2016, 2023], (11.556365, 104.92865))
('Denpasar', [2016, 2023], (-8.673105, 115.233856))
('Davao', [2016, 2023], (7.070

In [13]:
def main():
    """Process multiple cities and years"""
    
    # Define cities with their years and coordinates
    # You need to modify this list according to your actual cities
    
    # Process each city and year
    results = []
    for city_name, years, city_center in cities:
        for year in years:
            # Construct TIFF file path
            tiff_file = f"images/{city_name}_{year}.tif"
            
            # Check if file exists
            if not os.path.exists(tiff_file):
                print(f"[WARNING] TIFF file not found: {tiff_file}")
                continue
            
            # Process city (removed min_confidence parameter)
            success = process_city(tiff_file, city_name, year,  city_center,t=0.5)
            results.append({
                "city": city_name,
                "year": year,
                "success": success
            })
    
    # Print summary
    print("\n[SUMMARY] Processing Results:")
    for result in results:
        status = "SUCCESS" if result["success"] else "FAILED"
        print(f"  {result['city']} {result['year']}: {status}")

if __name__ == "__main__":
    main()

[INFO] Processing Mumbai 2016 with 3x3 sliding window analysis
[DEBUG] Reading TIFF file...
[DEBUG] Reading TIFF file: images/Mumbai_2016.tif
[DEBUG] Bands: 3
[DEBUG] Shape: (3, 4002, 4001)
[DEBUG] CRS: EPSG:4326
[DEBUG] Bounds: BoundingBox(left=72.63939084141592, bottom=18.789252314172323, right=72.99880678659214, top=19.148758090876957)
[DEBUG] Creating sliding window DataFrame with 3x3 windows using vectorized operations
[DEBUG] Image shape: (4002, 4001)
[DEBUG] Building presence range: 0.0 to 1.0
[DEBUG] Building existence pixels: 512123 out of 16012002
[DEBUG] Starting vectorized 3x3 window processing...
[DEBUG] Vectorized window processing completed
[DEBUG] Converting pixel coordinates to geographic coordinates...
[DEBUG] Calculating distances to city center...
[DEBUG] Created DataFrame with 15996000 windows
[INFO] Window analysis statistics:
  - Building count range: 0 to 9
  - Average height range: 0.00 to 99.46
  - Distance range: 0.00 to 27.49 km
  - Windows with buildings: 1

# Unused functions

## Alpha Shape

In [ ]:
def create_alpha_shape(points, source_crs, alpha_meters=None, bw=None):
    """Create alpha shape from points using approach adapted from R"""
    print(f"[DEBUG] Creating alpha shape")
    start_time = time.time()
    
    try:
        # Create a DataFrame from the points for sampling
        df = pd.DataFrame(points, columns=['x', 'y'])
        
        # Sample points (equivalent to R's sample.int)
        sample_size = min(df.shape[0], 10000)
        if sample_size < df.shape[0]:
            v = np.random.choice(df.shape[0], size=sample_size, replace=False)
            subset = df.iloc[v]
        else:
            subset = df
        
        # Get unique x,y pairs
        u = subset[['x', 'y']].drop_duplicates()
        
        # Add small random noise (as in the R code)
        u_with_noise = u.values + np.random.normal(0, 1/1000, u.shape)
        
        print(f"[DEBUG] Computing alpha shape with {len(u_with_noise)} points")
        
        # Convert to projected coordinates - pass the coordinates directly, not Point objects
        projected_points, utm_crs = convert_to_projected_coords(u_with_noise, source_crs)
        
        # Compute alpha shape
        # If bw is provided, use it to calculate alpha as in R
        if bw is not None:
            alpha = 3 * bw
            print(f"[DEBUG] Using R-style alpha calculation: alpha = {alpha}")
        else:
            alpha = alpha_meters
            print(f"[DEBUG] Using provided alpha: alpha = {alpha}")
        
        alpha_shape = alphashape.alphashape(projected_points, alpha)
        
        # Create a GeoDataFrame with the alpha shape
        gdf_alpha = gpd.GeoDataFrame(geometry=[alpha_shape], crs=utm_crs)
        
        # Project back to original CRS
        gdf_original = gdf_alpha.to_crs(source_crs)
        
        alpha_shape_original = gdf_original.geometry.iloc[0]
        
        end_time = time.time()
        print(f"[DEBUG] Alpha shape created in {end_time - start_time:.2f} seconds")
        
        return alpha_shape_original
    except Exception as e:
        print(f"[ERROR] Alpha shape creation failed: {e}")
        raise

def save_alpha_shape(alpha_shape, city_name, year, output_dir="alpha_shapes"):
    """Save alpha shape to a pickle file with city and year information"""
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    filename = os.path.join(output_dir, f"{city_name}_{year}_alpha_shape.pkl")
    print(f"[DEBUG] Saving alpha shape to {filename}")
    
    try:
        with open(filename, 'wb') as f:
            pickle.dump(alpha_shape, f)
        print("[DEBUG] Alpha shape saved successfully")
    except Exception as e:
        print(f"[ERROR] Failed to save alpha shape: {e}")

def load_alpha_shape(city_name, year, input_dir="alpha_shapes"):
    """Load alpha shape from a pickle file with city and year information"""
    filename = os.path.join(input_dir, f"{city_name}_{year}_alpha_shape.pkl")
    print(f"[DEBUG] Attempting to load alpha shape from {filename}")
    
    try:
        if os.path.exists(filename):
            with open(filename, 'rb') as f:
                alpha_shape = pickle.load(f)
            print("[DEBUG] Alpha shape loaded successfully")
            return alpha_shape
        else:
            print("[WARNING] Alpha shape file not found")
            return None
    except Exception as e:
        print(f"[ERROR] Failed to load alpha shape: {e}")
        return None

def filter_pixels_inside_alpha_hull(data, transform, alpha_hull, city_center, crs, threshold=0.34, output_dir="pixel_data"):
    """
    Filter pixels inside the alpha hull and calculate building metrics with efficient spatial indexing
    """
    print(f"[DEBUG] Efficiently filtering pixels inside alpha hull with threshold > {threshold}")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Extract bands
    building_height = data[1]  # Height band (band 2)
    building_presence = data[2]  # Presence band (band 3)

    # Create a mask for pixels with minimum presence
    presence_mask = building_presence > 0

    # Get coordinates of all potential pixels in one batch operation
    rows, cols = np.where(presence_mask)

    print(f"[DEBUG] Found {len(rows)} potential pixels with presence > 0")
    print("[DEBUG] Converting to coordinates...")

    # Fast batch conversion of pixel indices to coordinates
    xs, ys = xy(transform, rows, cols, offset='center')

    # Create points DataFrame
    points_df = pd.DataFrame({
        'row': rows,
        'col': cols,
        'longitude': xs,
        'latitude': ys
    })

    # Create GeoDataFrame with Point geometries
    print("[DEBUG] Creating GeoDataFrame for spatial filtering...")
    geometry = [Point(x, y) for x, y in zip(xs, ys)]
    gdf = gpd.GeoDataFrame(points_df, geometry=geometry, crs=crs)

    # Efficient spatial query to find points inside the alpha hull
    print("[DEBUG] Performing spatial join...")
    alpha_gdf = gpd.GeoDataFrame(geometry=[alpha_hull], crs=crs)
    points_inside = gpd.sjoin(gdf, alpha_gdf, predicate='within')

    print(f"[DEBUG] Found {len(points_inside)} points inside alpha hull")

    # Now get the building data for these points and calculate metrics
    print("[DEBUG] Calculating building metrics...")

    # Extract rows and columns for points inside the hull
    inside_rows = points_inside['row'].values
    inside_cols = points_inside['col'].values

    # Batch extract values
    presence_values = building_presence[inside_rows, inside_cols]
    height_values = building_height[inside_rows, inside_cols]

    # Apply threshold to determine building existence
    building_exists = (presence_values > threshold).astype(int)

    # Calculate building volumes
    building_volumes = height_values * building_exists

    # Calculate distances to city center
    # Convert the coordinates to arrays for vectorized haversine calculation
    lat_array = points_inside['latitude'].values
    lon_array = points_inside['longitude'].values

    # Vectorized haversine calculation
    lat1, lon1 = np.radians(lat_array), np.radians(lon_array)
    lat2, lon2 = np.radians(city_center[0]), np.radians(city_center[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distances = c * 6371  # Earth radius in km

    # Create DataFrame with all metrics
    df_pixels = pd.DataFrame({
        'longitude': points_inside['longitude'].values,
        'latitude': points_inside['latitude'].values,
        'row': inside_rows,
        'col': inside_cols,
        'building_presence': presence_values,
        'building_height': height_values,
        'building_exist': building_exists,
        'building_volume': building_volumes,
        'distance_to_center': distances
    })

    print(f"[DEBUG] Processed {len(df_pixels)} pixels inside the alpha hull")

    return df_pixels

## Volume Kernel Density Estimation

In [ ]:
def generate_volume_density_map(df, city_name, year, bw=None, output_dir="density_maps"):
    """Volume density map function using pixel data"""
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Verify DataFrame has data
    if df.empty:
        print("[ERROR] Cannot generate density map - empty DataFrame")
        return

    # Verify enough data points
    if len(df) < 2:
        print("[ERROR] Insufficient data points for kernel density estimation")
        return

    print("[DEBUG] Computing volume kernel density")

    try:
        # Use logarithmic scaling for better visualization
        log_volumes = np.log1p(df['building_volume'])

        # Kernel density estimation
        print(f"[DEBUG] Preparing density estimation")
        print(f"  Volume data points: {len(df)}")
        print(f"  Log volumes shape: {log_volumes.shape}")

        # Use 2D coordinates for density estimation
        coords = df[['longitude', 'latitude']].values

        print(f"[DEBUG] Coordinates shape: {coords.shape}")

        # Create grid for density map
        x_grid, y_grid = np.meshgrid(
            np.linspace(df['longitude'].min(), df['longitude'].max(), 200),
            np.linspace(df['latitude'].min(), df['latitude'].max(), 200)
        )

        # Kernel density estimation with R-style bandwidth if provided
        if bw is not None:
            print(f"[DEBUG] Using R-style bandwidth: {bw} for KDE")
            # Use Scott's rule to adjust bandwidth
            scott_factor = len(coords)**(-1/6)
            # Convert geographical bandwidth to KDE bandwidth parameter
            kde_bandwidth = bw * scott_factor
            kde = gaussian_kde(coords.T, weights=log_volumes, bw_method=kde_bandwidth)
        else:
            # Use default bandwidth
            kde = gaussian_kde(coords.T, weights=log_volumes)

        # Compute density
        grid_coords = np.vstack([x_grid.ravel(), y_grid.ravel()])
        density_values = kde(grid_coords).reshape(x_grid.shape)

        # Save the density values and grid coordinates
        density_data = {
            'x_grid': x_grid,
            'y_grid': y_grid,
            'density_values': density_values,
            'bandwidth': kde.factor if bw is None else kde_bandwidth
        }
        density_output_file = os.path.join(output_dir, f"{city_name}_{year}_density_data.pkl")
        with open(density_output_file, 'wb') as f:
            pickle.dump(density_data, f)
        print(f"[DEBUG] Density data saved to: {density_output_file}")

        # Plot density map with contours
        plt.figure(figsize=(12, 10))

        # Plot density as a heat map
        im = plt.imshow(density_values, extent=[x_grid.min(), x_grid.max(), y_grid.min(), y_grid.max()],
                   origin='lower', cmap='viridis', alpha=0.7)

        # Add contours
        contours = plt.contour(x_grid, y_grid, density_values,
                              levels=10, colors='white', alpha=0.8, linewidths=0.5)
        plt.clabel(contours, inline=True, fontsize=8, fmt='%.2f')

        plt.colorbar(im, label='Log Volume Density')
        plt.title(f'Building Volume Density Map - {city_name} {year}')
        plt.xlabel('Longitude')
        plt.ylabel('Latitude')

        # Save figure
        output_file = os.path.join(output_dir, f"{city_name}_{year}_volume_density_map.png")
        plt.savefig(output_file)
        plt.close()

        print(f"[DEBUG] Volume density map generated successfully: {output_file}")

    except Exception as e:
        print(f"[ERROR] Volume density map generation failed: {e}")
        import traceback
        traceback.print_exc()

## Visualize relationship

In [ ]:
def visualize_relationships(df, city_name, year, output_dir="visualizations"):
    """Create scatter plots for building metrics vs distance to center"""
    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Ensure DataFrame is not empty
    if df.empty:
        print("[ERROR] Cannot visualize empty DataFrame")
        return
    
    # Verify required columns exist
    required_columns = ['distance_to_center', 'building_exist', 'building_height', 'building_volume']
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        print(f"[ERROR] Missing columns: {missing_columns}")
        print("Available columns:", list(df.columns))
        return
    
    # Create multi-panel plot
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    
    # Scatter plots
    metrics = ['building_exist', 'building_height', 'building_volume']
    for i, metric in enumerate(metrics):
        # Metric vs Distance
        axs[0, i].scatter(df['distance_to_center'], df[metric], alpha=0.5, s=1)
        axs[0, i].set_title(f'{metric.capitalize()} vs Distance to Center')
        axs[0, i].set_xlabel('Distance to City Center (km)')
        axs[0, i].set_ylabel(metric.capitalize())
    
    # Metric distribution
    for i, metric in enumerate(metrics):
        axs[1, i].hist(df[metric], bins=30)
        axs[1, i].set_title(f'{metric.capitalize()} Distribution')
        axs[1, i].set_xlabel(metric.capitalize())
        axs[1, i].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.suptitle(f'Building Metrics Analysis - {city_name} {year}', y=1.02)
    
    # Save figure
    output_file = os.path.join(output_dir, f"{city_name}_{year}_building_metrics_analysis.png")
    plt.savefig(output_file)
    plt.close()
    
    print(f"[DEBUG] Relationship visualization completed: {output_file}")

## How to Call Alpha shape & Kernel (previous version)

In [1]:
def process_city(tiff_file, city_name, year, city_center, threshold=0.34):
    """Process a single city's TIFF file"""
    print(f"[INFO] Processing {city_name} {year} with threshold {threshold}")
    
    try:
        # Create output directories
        output_dirs = ["alpha_shapes", "pixel_data", "density_maps", "visualizations"]
        for directory in output_dirs:
            os.makedirs(directory, exist_ok=True)
        
        # Read TIFF
        print("[DEBUG] Reading TIFF file...")
        result = read_tiff(tiff_file)
        if result is not None:
            data, meta, transform, crs, bounds = result
            # Proceed with processing the data
        else:
            # Handle the error case
            print("Failed to read the TIFF file.")
            return False
        
        # Create full pixels DataFrame (add this line)
        df_all_pixels = create_full_pixels_dataframe(data, transform, city_center, crs, threshold=threshold)
        
        # Save the all pixels dataframe with city and year info (add this line)
        all_pixels_output_file = os.path.join("pixel_data", f"{city_name}_{year}_all_pixels_data.csv")
        df_all_pixels.to_csv(all_pixels_output_file, index=False)
        print(f"[DEBUG] Saved {len(df_all_pixels)} total pixels to {all_pixels_output_file}")
        
        # # Continue with the rest of your processing...
        # # Extract building points and data
        # print("[DEBUG] Extracting building points...")
        # points, heights, rows, cols = extract_building_points(data, transform, threshold=threshold)
        
        # # Extract building points and data
        # print("[DEBUG] Extracting building points...")
        # points, heights, rows, cols = extract_building_points(data, transform, threshold=threshold)
        
        # # Create DataFrame from points for calculating bandwidth
        # df_points = pd.DataFrame(points, columns=['x', 'y'])
        
        # # Calculate bandwidth as in the R code
        # bw = 0.001/np.cos(np.mean(df_points['y'])/90*np.pi/2)
        # print(f"[DEBUG] Calculated bandwidth: {bw}")
        
        # # Try to load existing alpha shape
        # alpha_hull = load_alpha_shape(city_name, year)
        
        # # If no existing alpha shape, create and save one
        # if alpha_hull is None:
        #     # Create alpha hull using R-style approach with calculated bw
        #     print("[DEBUG] Creating alpha shape...")
        #     alpha_hull = create_alpha_shape(points, crs, bw=bw)
            
        #     # Save the newly created alpha shape
        #     save_alpha_shape(alpha_hull, city_name, year)
        
        # # Filter pixels inside alpha hull and calculate metrics
        # print("[DEBUG] Filtering pixels inside alpha hull...")
        # df_pixels = filter_pixels_inside_alpha_hull(data, transform, alpha_hull, city_center, crs, threshold=threshold)
        
        # # Save pixel data
        # pixel_output_file = os.path.join("pixel_data", f"{city_name}_{year}_pixel_data.csv")
        # df_pixels.to_csv(pixel_output_file, index=False)
        # print(f"[DEBUG] Saved {len(df_pixels)} pixels to {pixel_output_file}")
        
        # # Visualize relationships
        # print("[DEBUG] Visualizing relationships...")
        # visualize_relationships(df_pixels, city_name, year)
        
        # # Generate volume density map with contours (using R-style bw)
        # # print("[DEBUG] Generating volume density map...")
        # # generate_volume_density_map(df_pixels, city_name, year, bw=bw)
        
        # # Visualize alpha hull
        # gdf_hull = gpd.GeoDataFrame(geometry=[alpha_hull], crs=crs)
        # plt.figure(figsize=(12, 10))
        # gdf_hull.plot(alpha=0.5, edgecolor='red')
        # plt.title(f'{city_name} {year} Building Alpha Hull')
        # plt.xlabel('Longitude')
        # plt.ylabel('Latitude')
        # hull_output_file = os.path.join("visualizations", f"{city_name}_{year}_alpha_hull.png")
        # plt.savefig(hull_output_file)
        # plt.close()
        
        print(f"[INFO] {city_name} {year} analysis completed successfully!")
        return True
        
    except Exception as e:
        print(f"[ERROR] An error occurred while processing {city_name} {year}: {e}")
        import traceback
        traceback.print_exc()
        return False

In [ ]:
def main():
    """Process multiple cities and years"""
    # Define cities and their data
    # Format: [city_name, [years], (city_center_lat, city_center_lon)]
    cities = [
        # City name, list of years, (latitude, longitude) of city center
        #("Abidjan", [2016, 2023], (5.3252258, -4.019603))
        # ("Antananarivo", [2016, 2023], (-18.9155584, 47.5216452))
        # ("Yaounde", [2016, 2023], (3.8614776, 11.5191364))
        ("Lusaka", [2016, 2023], (-15.4191874, 28.3054202))
        # Add more cities as needed
    ]
    
    # Default threshold for building presence
    threshold = 0.34
    
    # Process each city and year
    results = []
    for city_name, years, city_center in cities:
        for year in years:
            # Construct TIFF file path
            tiff_file = f"{city_name}_{year}.tif"
            
            # Check if file exists
            if not os.path.exists(tiff_file):
                print(f"[WARNING] TIFF file not found: {tiff_file}")
                continue
            
            # Process city with specified threshold
            success = process_city(tiff_file, city_name, year, city_center, threshold=threshold)
            results.append({
                "city": city_name,
                "year": year,
                "success": success
            })
    
    # Print summary
    print("\n[SUMMARY] Processing Results:")
    for result in results:
        status = "SUCCESS" if result["success"] else "FAILED"
        print(f"  {result['city']} {result['year']}: {status}")

if __name__ == "__main__":
    main()

# 提取全部像素数据（保留confidence level）

In [ ]:
import os
import pickle
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
import cupy as cp
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import gaussian_kde
from shapely.geometry import Point, Polygon
import alphashape
from scipy.spatial.distance import cdist
import pyproj
from shapely.ops import transform
from functools import partial
import math
from pathlib import Path
from rasterio.transform import xy  # Import the xy function

def read_tiff(file_path):
    """Read TIFF file with enhanced debugging"""
    print(f"[DEBUG] Reading TIFF file: {file_path}")
    try:
        with rasterio.open(file_path) as src:
            data = src.read()
            meta = src.meta.copy()
            transform = src.transform
            crs = src.crs
            bounds = src.bounds  # Additional metadata

            print(f"[DEBUG] Bands: {src.count}")
            print(f"[DEBUG] Shape: {data.shape}")
            print(f"[DEBUG] CRS: {crs}")
            print(f"[DEBUG] Bounds: {bounds}")

            return data, meta, transform, crs, bounds
    except RasterioIOError as e:
        print(f"[ERROR] TIFF reading failed: {e}")
        return None

def convert_to_projected_coords(points, source_crs):
    """
    Convert points from source CRS to a suitable UTM projection
    
    Parameters:
    -----------
    points : numpy.ndarray
        Array of shape (n, 2) containing x,y coordinates
    source_crs : str or CRS
        The source coordinate reference system
    
    Returns:
    --------
    projected_points : list
        List of projected coordinates as tuples
    utm_crs : CRS
        The target UTM CRS
    """
    # Check if points array is empty using NumPy's size attribute
    if points.size == 0:
        return [], None
        
    # Convert source_crs to pyproj CRS if it's a string
    if isinstance(source_crs, str):
        source_crs = pyproj.CRS.from_string(source_crs)
    
    # Calculate the centroid of points to determine the appropriate UTM zone
    mean_lon = np.mean(points[:, 0])
    mean_lat = np.mean(points[:, 1])
    
    # Determine UTM zone based on the centroid
    utm_crs = pyproj.CRS.from_dict({
        'proj': 'utm',
        'zone': int((mean_lon + 180) / 6) + 1,
        'south': mean_lat < 0
    })
    
    # Create the transformer
    transformer = pyproj.Transformer.from_crs(source_crs, utm_crs, always_xy=True)
    
    # Transform all points
    x, y = transformer.transform(points[:, 0], points[:, 1])
    
    # Convert to a list of tuples for alphashape
    projected_points = list(zip(x, y))
    
    return projected_points, utm_crs

def extract_building_points(data, transform, min_confidence=0.0):
    """Extract building points with GPU acceleration - now outputs confidence levels directly"""
    print(f"[DEBUG] Extracting building points with minimum confidence > {min_confidence}")
    building_height = data[1]    # Assuming 2nd band is height
    building_confidence = data[2]  # Assuming 3rd band is confidence level
    
    # Use CuPy for faster processing
    try:
        building_confidence_gpu = cp.array(building_confidence)
        building_height_gpu = cp.array(building_height)
        
        # Find valid indices (only filter by minimum confidence if needed)
        valid_indices = cp.where(building_confidence_gpu > min_confidence)
        rows, cols = valid_indices[0].get(), valid_indices[1].get()
    except Exception:
        # Fallback to NumPy
        valid_indices = np.where(building_confidence > min_confidence)
        rows, cols = valid_indices[0], valid_indices[1]
    
    # Extract coordinates, heights, and confidence levels
    points = []
    heights = []
    confidences = []
    
    for i in tqdm(range(len(rows)), desc="Converting pixels"):
        row, col = rows[i], cols[i]
        x, y = rasterio.transform.xy(transform, row, col)
        points.append((x, y))
        heights.append(building_height[row, col])
        confidences.append(building_confidence[row, col])
    
    return points, heights, confidences, rows, cols


def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate the Haversine distance between two points on Earth"""
    # Convert coordinates to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of Earth in kilometers
    r = 6371
    
    return c * r

def create_full_pixels_dataframe(data, transform, city_center, crs, min_confidence=0.0, output_dir="pixel_data"):
    """
    Create a DataFrame containing all pixels with building confidence levels and height data
    No longer applies threshold - outputs raw confidence values
    """
    print(f"[DEBUG] Creating full pixels DataFrame with minimum confidence = {min_confidence}")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Extract bands
    building_height = data[1]  # Height band (band 2)
    building_confidence = data[2]  # Confidence band (band 3)

    # Get coordinates of all pixels
    rows, cols = np.indices(building_confidence.shape)
    rows = rows.flatten()
    cols = cols.flatten()

    print(f"[DEBUG] Processing {len(rows)} total pixels")

    # Fast batch conversion of pixel indices to coordinates
    xs, ys = xy(transform, rows, cols, offset='center')

    # Batch extract values
    confidence_values = building_confidence[rows, cols]
    height_values = building_height[rows, cols]



    # Calculate distances to city center
    lat_array = ys
    lon_array = xs

    lat1, lon1 = np.radians(lat_array), np.radians(lon_array)
    lat2, lon2 = np.radians(city_center[0]), np.radians(city_center[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distances = c * 6371  # Earth radius in km

    # Create DataFrame with all metrics - raw confidence and height only
    df_all_pixels = pd.DataFrame({
        'longitude': lon_array,
        'latitude': lat_array,
        'row': rows,
        'col': cols,
        'building_confidence': confidence_values,  # Raw confidence level (0-1)
        'building_height': height_values,
        'distance_to_center': distances
    })

    return df_all_pixels


def process_city(tiff_file, city_name, year, city_center, min_confidence=0.0):
    """Process a single city's TIFF file - now outputs confidence levels directly"""
    print(f"[INFO] Processing {city_name} {year} with minimum confidence {min_confidence}")
    
    try:
        # Create output directories
        output_dirs = ["pixel_data", "visualizations"]
        for directory in output_dirs:
            os.makedirs(directory, exist_ok=True)
        
        # Read TIFF
        print("[DEBUG] Reading TIFF file...")
        result = read_tiff(tiff_file)
        if result is not None:
            data, meta, transform, crs, bounds = result
            # Proceed with processing the data
        else:
            # Handle the error case
            print("Failed to read the TIFF file.")
            return False
        
        # Create full pixels DataFrame with confidence levels
        df_all_pixels = create_full_pixels_dataframe(data, transform, city_center, crs, min_confidence=min_confidence)
        
        # Save the all pixels dataframe with city and year info
        all_pixels_output_file = os.path.join("pixel_data", f"{city_name}_{year}_all_pixels_confidence.csv")
        df_all_pixels.to_csv(all_pixels_output_file, index=False)
        print(f"[DEBUG] Saved {len(df_all_pixels)} total pixels to {all_pixels_output_file}")
        
        # Print confidence statistics
        confidence_stats = df_all_pixels['building_confidence'].describe()
        print(f"[INFO] Building confidence statistics:")
        print(confidence_stats)
        
        print(f"[INFO] {city_name} {year} analysis completed successfully!")
        return True
        
    except Exception as e:
        print(f"[ERROR] An error occurred while processing {city_name} {year}: {e}")
        import traceback
        traceback.print_exc()
        return False

In [ ]:
def main():
    """Process multiple cities and years"""
    
    # Default minimum confidence for filtering (0.0 means no filtering)
    min_confidence = 0.0
    
    # Process each city and year
    results = []
    for city_name, years, city_center in cities:
        for year in years:
            # Construct TIFF file path
            tiff_file = f"images/{city_name}_{year}.tif"
            
            # Check if file exists
            if not os.path.exists(tiff_file):
                print(f"[WARNING] TIFF file not found: {tiff_file}")
                continue
            
            # Process city with specified minimum confidence
            success = process_city(tiff_file, city_name, year, city_center, min_confidence=min_confidence)
            results.append({
                "city": city_name,
                "year": year,
                "success": success
            })
    
    # Print summary
    print("\n[SUMMARY] Processing Results:")
    for result in results:
        status = "SUCCESS" if result["success"] else "FAILED"
        print(f"  {result['city']} {result['year']}: {status}")

if __name__ == "__main__":
    main()